In [ ]:
from preprocessing import * 
import mne
import pywt
from sklearn import svm
from sklearn import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy import stats

%load_ext autoreload
%autoreload 2

In [ ]:
x = np.load('numpy_data/A01TX.npy')
x = x[:, :,1:]
y = np.load('numpy_data/A01TY.npy')

In [ ]:
x = np.swapaxes(x, 1, 2)
print(x.shape, y.shape)


In [65]:
def featurize(x):
  coeff = pywt.wavedec(x, 'db4', level = 7)
  return coeff
  # features = np.zeros((*x.shape[0:-1], 6*7))
  
  # for i, c in enumerate(coeff):
  #   mean = np.mean(c, axis=-1)
  #   abs_mean = np.mean(np.abs(c), axis=-1)
  #   mean_squared = np.mean(c**2, axis=-1)
  #   std = np.std(c, axis=-1)
  #   var = np.var(c, axis=-1)
  #   skewness = stats.skew(c, axis=-1)

  #   if len(features.shape) == 3:
  #     features[:,:, i*6: i*6 + 6] = np.stack([mean, abs_mean, mean_squared, std, var, skewness], axis = -1)
  #   elif len(features.shape) == 2:
  #     features[:, i*6: i*6 + 6] = np.stack([mean, abs_mean, mean_squared, std, var, skewness], axis = -1)

  # return features.reshape((*features.shape[0:-2], 42*25))

coeff = featurize(x)
print(len(coeff))
# print(x.shape, featurize(x).shape)

8


c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 7 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [66]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
csp = [mne.decoding.CSP() for _ in range(len(coeff))]

coeff = featurize(X_train)
x_train = np.concatenate(tuple(csp[x].fit_transform(coeff[x][:,:,:], y_train) for x  in range(len(coeff))),axis=-1)

Computing rank from data with rank=None
    Using tolerance 57 (2.2e-16 eps * 25 dim * 1e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 71 (2.2e-16 eps * 25 dim * 1.3e+16  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 53 (2.2e-16 eps * 25 dim * 9.5e+15  max singular value)
    Estimated rank (mag): 25
    MAG: rank 25 computed from 25 data channels with 0 projectors
Reducing data rank from 25 -> 25
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 43 (2.2e-16 eps * 25 dim * 7.7e+15  max singular value)
    Estimated rank (ma

In [67]:
x_train.shape, y_train.shape
coeff = featurize(X_test)
x_test = np.concatenate(tuple(csp[x].transform(coeff[x][:,:,:]) for x  in range(len(coeff))),axis=-1)

In [68]:

# now we have CA6 + CD6 -> CD1 (6 matrices) * 7 features = 42 feature vector
clf = pipeline.make_pipeline(StandardScaler(), svm.SVC())
clf.fit(x_train, y_train)


Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [69]:
print(clf.predict(x_train))
print("Accuracy is ", sum(clf.predict(x_train) == y_train) / len(X_train))
print(len(x_train))

print(clf.predict(x_test))
print("test Accuracy is ", sum(clf.predict(x_test) == y_test) / len(X_test))
print(len(x_test))


# c = pywt.WaveletPacket(x[0,0,:], 'db4', mode='symmetric', maxlevel=5)

# for node in c.get_level(5, 'natural'):
  # print(node.path)



[4. 4. 4. 1. 1. 2. 4. 1. 3. 1. 4. 3. 4. 1. 1. 1. 2. 2. 4. 2. 4. 3. 4. 4.
 2. 4. 1. 4. 1. 4. 2. 3. 4. 3. 3. 2. 1. 1. 3. 3. 3. 4. 1. 3. 2. 2. 2. 1.
 3. 2. 4. 2. 3. 3. 3. 1. 4. 3. 1. 4. 4. 1. 2. 2. 3. 2. 1. 2. 2. 1. 1. 3.
 1. 2. 2. 3. 3. 4. 3. 2. 3. 1. 4. 3. 4. 4. 2. 1. 3. 4. 4. 1. 3. 2. 4. 4.
 1. 3. 2. 3. 3. 4. 1. 1. 2. 1. 4. 2. 2. 4. 3. 4. 3. 3. 3. 1. 1. 4. 2. 2.
 3. 3. 1. 1. 1. 1. 1. 1. 3. 3. 4. 1. 1. 2. 2. 1. 1. 2. 2. 1. 2. 2. 1. 4.
 4. 1. 4. 4. 3. 2. 2. 2. 2. 4. 3. 2. 4. 2. 2. 4. 1. 2. 2. 3. 4. 2. 2. 2.
 4. 1. 4. 2. 2. 2. 2. 4. 3. 2. 2. 4. 3. 3. 2. 1. 3. 1. 2. 2. 2. 2. 2. 4.
 4. 2. 2. 4. 1. 2. 2. 3. 2. 4. 4. 3. 1. 1. 3. 3. 1. 3. 4. 1. 1. 4. 1. 2.
 4. 4. 4. 2. 4. 1. 4. 3. 3. 1. 3. 2. 4. 1.]
Accuracy is  0.9521739130434783
230
[4. 4. 2. 2. 1. 3. 3. 1. 1. 3. 4. 3. 1. 4. 4. 3. 2. 3. 3. 4. 3. 2. 3. 2.
 2. 2. 1. 1. 1. 3. 3. 2. 4. 3. 2. 2. 2. 4. 3. 2. 2. 3. 3. 2. 3. 3. 4. 3.
 1. 1. 3. 3. 3. 1. 4. 2. 3. 1.]
test Accuracy is  0.6724137931034483
58


In [71]:
pywt.families()
pywt.wavelist()

['bior1.1',
 'bior1.3',
 'bior1.5',
 'bior2.2',
 'bior2.4',
 'bior2.6',
 'bior2.8',
 'bior3.1',
 'bior3.3',
 'bior3.5',
 'bior3.7',
 'bior3.9',
 'bior4.4',
 'bior5.5',
 'bior6.8',
 'cgau1',
 'cgau2',
 'cgau3',
 'cgau4',
 'cgau5',
 'cgau6',
 'cgau7',
 'cgau8',
 'cmor',
 'coif1',
 'coif2',
 'coif3',
 'coif4',
 'coif5',
 'coif6',
 'coif7',
 'coif8',
 'coif9',
 'coif10',
 'coif11',
 'coif12',
 'coif13',
 'coif14',
 'coif15',
 'coif16',
 'coif17',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'db6',
 'db7',
 'db8',
 'db9',
 'db10',
 'db11',
 'db12',
 'db13',
 'db14',
 'db15',
 'db16',
 'db17',
 'db18',
 'db19',
 'db20',
 'db21',
 'db22',
 'db23',
 'db24',
 'db25',
 'db26',
 'db27',
 'db28',
 'db29',
 'db30',
 'db31',
 'db32',
 'db33',
 'db34',
 'db35',
 'db36',
 'db37',
 'db38',
 'dmey',
 'fbsp',
 'gaus1',
 'gaus2',
 'gaus3',
 'gaus4',
 'gaus5',
 'gaus6',
 'gaus7',
 'gaus8',
 'haar',
 'mexh',
 'morl',
 'rbio1.1',
 'rbio1.3',
 'rbio1.5',
 'rbio2.2',
 'rbio2.4',
 'rbio2.6',
 'rbio2.8',
 'rbio3.1',